In [1]:
import os
import uuid

from flask import Flask, request, jsonify
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

model_used = "mog_model_1_4.743589398682443.h5"

app = Flask(__name__)

2022-11-05 17:46:05.200924: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-05 17:46:05.297072: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-05 17:46:05.300642: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-05 17:46:05.300652: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
df  = pd.read_csv(r"auto-mpg.csv", na_values=["NA", "?"])
cols = [x for x in df.columns if x not in ("mpg", "car name")]
expected = {}
for i, name in enumerate(cols):
    expected[name] = {}
    expected[name]["min"] = df[name].min() 
    expected[name]["max"] = df[name].max()
# for key, _ in expected.items():
#     print(key)

In [3]:
model = load_model(os.path.join(os.getcwd(), model_used))
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 25)                200       
                                                                 
 dense_23 (Dense)            (None, 10)                260       
                                                                 
 dense_24 (Dense)            (None, 1)                 11        
                                                                 
Total params: 471
Trainable params: 471
Non-trainable params: 0
_________________________________________________________________


2022-11-05 17:46:06.330565: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-05 17:46:06.330583: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-05 17:46:06.330596: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (NumericalCatalyst): /proc/driver/nvidia/version does not exist
2022-11-05 17:46:06.330769: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
@app.route("/api", methods=["POST"])
def mpg_prediction():
    content = request.json
    errors = []
    for name in content:
        if name in expected:
            expected_min = expected[name]["min"]
            expected_max = expected[name]["max"]
            value = content[name]
            if value < expected_min or value > expected_max:
                errors.append(f"{name} out of bounds: {expected_min} and {expected_max}")
        else:
            errors.append(f"Unexpected field: {name}")
    for name in expected:
        if name not in content:
            errors.append(f"Missing value: {name}")
    if len(errors) < 1:
        x = np.zeros((1, 7))
        x[0,0] = content["cylinders"]
        x[0,1] = content["displacement"]
        x[0,2] = content["horsepower"]
        x[0,3] = content["weight"]
        x[0,4] = content["acceleration"]
        x[0,5] = content["model year"]
        x[0,6] = content["origin"]

        prediction = model.predict(x)
        mpg = float(prediction[0])
        response = {"id": str(uuid.uuid4()), "mpg": mpg, "errors": errors}
    else:
        response = {"id": str(uuid.uuid4()), "errors": errors}
    return jsonify(response)

In [5]:
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


1/1 [==============================] - 0s 91ms/step


127.0.0.1 - - [05/Nov/2022 17:46:09] "POST /api HTTP/1.1" 200 -


1/1 [==============================] - 0s 29ms/step


...

127.0.0.1 - - [05/Nov/2022 17:47:47] "POST /api HTTP/1.1" 200 -


1/1 [==============================] - 0s 24ms/step


127.0.0.1 - - [05/Nov/2022 17:47:55] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2022 17:48:04] "POST /api HTTP/1.1" 200 -


1/1 [==============================] - 0s 24ms/step


127.0.0.1 - - [05/Nov/2022 17:48:27] "POST /api HTTP/1.1" 200 -
